In [ ]:
import openai, json

client = openai.OpenAI()
messages = []

In [26]:
def get_weather(city):
    return '33 dregrees Celsius'

FUNCTION_MAP = {
    'get_weather': get_weather
}

In [27]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function that gets the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather of",
                    },
                  
                },
                "required": ["city"],
            }
        }
    }
]

In [ ]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls > 0:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [{
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments
                }
            } for tool_call in message.tool_calls]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function {function_name} with arguments {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError as e:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments) if function_to_run else None

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })

In [ ]:
def call_ai():
     response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
     )
     process_ai_response(response.choices[0].message)
     print(response)
     message = response.choices[0].message.content
     messages.append({"role": "assistant", "content": message})
     print(f"Ai: {message}")

In [29]:
while True:
    message = input("Send a message to the LLM...")
    if message == 'quit' or message == 'q':
        break
    else :
        messages.append({"role": "user", "content": message})
        print(f"user: {message}")
        call_ai()

user: good
ChatCompletion(id='chatcmpl-CFiSoKQrGQQK1EDET38Nd1xVKGZAV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Glad to hear that! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757861874, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=13, prompt_tokens=57, total_tokens=70, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Ai: Glad to hear that! How can I assist you today?
